The idea of coupled recursion is combining multiple recursion processes together in a way they can use each others recursion calls to reduce variance. This is path reuse like. In the example of period $2$ the next point position was independent of the current one (always $\text{Uniform(0,1)}$). Coupled recursion in that case is similar to splitting but instead of making new recursion calls you reuse the recursion calls from the coupled processes. <br>

This works well when working with only $1$ Green function. The implementation of coupled processes can be  thought of a "recursion" vector where each recursion all recursive calls get shared.



An good example of coupled recursion is "Path replay backpropagation" like in Solving Inverse PDE Problems using Grid-Free Monte Carlo Estimators (we aren't sure if this is like tree walking from Monte Carlo Geometry Processing: A Grid-Free Approach to PDE-Based Methods on Volumetric Domains). Let's work out the same example as in local model via SALT but this time with a estimator of local information obtained via pathwise differentiation:

$$
y'=ay,y(0)=1,a=1
$$ 
This has as solution $y(t,a) = e^{at}$ and is equivalent with following integral equation:
$$
y(t,a) = 1 + \int_{0}^{t} a y(s,a)ds .
$$ 
Differentiating this in $a$ gives
$$
\partial_{a} y(t,a) = \int_{0}^{t} y(s,a) + a \partial_{a} y(t,a) ds .
$$ 
if you simulate this last expression you get a branching estimator this can be avoided by russian roulette but also by combining it with the first equation and sharing recursions:

$$ 
\left(\begin{array}{l}
y(t, a) \\
\partial_a y(t, a)
\end{array}\right)=\left(\begin{array}{l}
1 \\
0
\end{array}\right)+\int_0^t\left(\begin{array}{l}
a y(s, a) \\
y(s, a)+a \partial_{a}y(s, a)
\end{array}\right) d s
$$
Turning that into a recursive estimator implemented in code it looks like this:

In [34]:
from random import random
from math import exp
import numpy as np

# E for estimator
def E(t,a)->np.array:
    if t<1 and random()>t:
        return np.array([1,0])
    e = E(random()*t,a)
    y = 1 + t*a*e[0] if t>1 else 1 + a*e[0]
    py= t*(e[0]+a*e[1]) if t>1 else e[0]+a*e[1]
    return np.array([y,py])
    
s = np.array([0.0,0.0])
nsim = 10**4
t,a = 0.5,2
for _ in range(nsim):
    s +=E(t,a)/nsim

sol = np.array([exp(t*a),t*exp(t*a)]) 
err = (s-sol)/sol
print(f"E = {s}")
print(f"%error is {err}")

E = [2.7276 1.3946]
%error is [0.00342796 0.02608934]


Lets look at a SDE instead of a ODE (we aren't sure if taking derivatives of SDE works):
$$
\begin{align*}
dX_t &= (1+\sigma X_{t})dW_{t} \\
d(\partial_{\sigma}X_{t}) &= (X_{t}+ \sigma\partial_{\sigma}X_{t} ) dW_t
\end{align*}
$$ 
